In [1]:
2+2

4

In [24]:
# Install required packages
!pip install torch torchvision
!pip install opencv-python-headless
!pip install pandas
!pip install pytesseract




In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Import necessary libraries
import os
import torch
import cv2
import pandas as pd
import pytesseract
from PIL import Image
import numpy as np

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [7]:

# Load annotations from CSV
annotations_csv = "/content/drive/MyDrive/Project/LPR/Licplatesdetection_train.csv"
annotations_df = pd.read_csv(annotations_csv)

# Define the image directory
images_dir = "/content/drive/MyDrive/Project/LPR/license_plates_detection_train"

# Create an empty list to store images
images = []

# Read images based on the image names in the CSV
for index, row in annotations_df.iterrows():
    img_name = row['img_id']
    img_path = os.path.join(images_dir, img_name)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Check if the image is successfully loaded
    if img is not None:
        images.append(img)
        print(f"Loaded {img_name}")
    else:
        print(f"Failed to load {img_name}")



Loaded 1.jpg
Loaded 2.jpg
Loaded 3.jpg
Loaded 4.jpg
Loaded 5.jpg
Loaded 6.jpg
Loaded 7.jpg
Loaded 8.jpg
Loaded 9.jpg
Loaded 10.jpg
Loaded 11.jpg
Loaded 12.jpg
Loaded 13.jpg
Loaded 14.jpg
Loaded 15.jpg
Loaded 16.jpg
Loaded 17.jpg
Loaded 18.jpg
Loaded 19.jpg
Loaded 20.jpg
Loaded 21.jpg
Loaded 22.jpg
Loaded 23.jpg
Loaded 24.jpg
Loaded 25.jpg
Loaded 26.jpg
Loaded 27.jpg
Loaded 28.jpg
Loaded 29.jpg
Loaded 30.jpg
Loaded 31.jpg
Loaded 32.jpg
Loaded 33.jpg
Loaded 34.jpg
Loaded 35.jpg
Loaded 36.jpg
Loaded 37.jpg
Loaded 38.jpg
Loaded 39.jpg
Loaded 40.jpg
Loaded 41.jpg
Loaded 42.jpg
Loaded 43.jpg
Loaded 44.jpg
Loaded 45.jpg
Loaded 46.jpg
Loaded 47.jpg
Loaded 48.jpg
Loaded 49.jpg
Loaded 50.jpg
Loaded 51.jpg
Loaded 52.jpg
Loaded 53.jpg
Loaded 54.jpg
Loaded 55.jpg
Loaded 56.jpg
Loaded 57.jpg
Loaded 58.jpg
Loaded 59.jpg
Loaded 60.jpg
Loaded 61.jpg
Loaded 62.jpg
Loaded 63.jpg
Loaded 64.jpg
Loaded 65.jpg
Loaded 66.jpg
Loaded 67.jpg
Loaded 68.jpg
Loaded 69.jpg
Loaded 70.jpg
Loaded 71.jpg
Loaded 72.jpg
L

In [8]:
# Define paths
annotations_csv = "/content/drive/MyDrive/Project/LPR/Licplatesdetection_train.csv"
images_dir = "/content/drive/MyDrive/Project/LPR/license_plates_detection_train"
preprocessed_dir = "/content/drive/MyDrive/Project/LPR/preprocessed_images"
labels_dir = "/content/drive/MyDrive/Project/LPR/Source/abels"

# Create directories if they don't exist
os.makedirs(preprocessed_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Define new image size
new_size = (640, 480)  # Example size, you can adjust it according to your requirements

# Function to convert bounding box coordinates to YOLO format
def convert_bbox_to_yolo_format(bbox, img_width, img_height):
    xmin, ymin, xmax, ymax = bbox
    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return (x_center, y_center, width, height)

# Iterate through annotations and preprocess images
for _, row in annotations_df.iterrows():
    img_name = row['img_id']
    img_path = os.path.join(images_dir, img_name)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Check if the image is successfully loaded
    if img is not None:
        # Resize the image
        img_resized = cv2.resize(img, new_size)

        # Save the preprocessed image
        preprocessed_img_path = os.path.join(preprocessed_dir, img_name)
        cv2.imwrite(preprocessed_img_path, img_resized)

        # Convert bounding box coordinates to YOLO format
        bbox = (row['xmin'], row['ymin'], row['xmax'], row['ymax'])
        yolo_bbox = convert_bbox_to_yolo_format(bbox, img.shape[1], img.shape[0])

        # Save YOLO format annotation
        label_file_path = os.path.join(labels_dir, img_name.replace(".jpg", ".txt"))
        with open(label_file_path, "w") as f:
            f.write(f"0 {' '.join(map(str, yolo_bbox))}\n")  # Assuming one class ('license_plate')

        print(f"Preprocessed and saved {img_name}")
    else:
        print(f"Failed to load {img_name}")


Preprocessed and saved 1.jpg
Preprocessed and saved 2.jpg
Preprocessed and saved 3.jpg
Preprocessed and saved 4.jpg
Preprocessed and saved 5.jpg
Preprocessed and saved 6.jpg
Preprocessed and saved 7.jpg
Preprocessed and saved 8.jpg
Preprocessed and saved 9.jpg
Preprocessed and saved 10.jpg
Preprocessed and saved 11.jpg
Preprocessed and saved 12.jpg
Preprocessed and saved 13.jpg
Preprocessed and saved 14.jpg
Preprocessed and saved 15.jpg
Preprocessed and saved 16.jpg
Preprocessed and saved 17.jpg
Preprocessed and saved 18.jpg
Preprocessed and saved 19.jpg
Preprocessed and saved 20.jpg
Preprocessed and saved 21.jpg
Preprocessed and saved 22.jpg
Preprocessed and saved 23.jpg
Preprocessed and saved 24.jpg
Preprocessed and saved 25.jpg
Preprocessed and saved 26.jpg
Preprocessed and saved 27.jpg
Preprocessed and saved 28.jpg
Preprocessed and saved 29.jpg
Preprocessed and saved 30.jpg
Preprocessed and saved 31.jpg
Preprocessed and saved 32.jpg
Preprocessed and saved 33.jpg
Preprocessed and sa

In [9]:
import os
from sklearn.model_selection import train_test_split

# Define the data directory
data_dir = "/content/drive/MyDrive/Project/LPR/preprocessed_images"

# Get the list of preprocessed image names
preprocessed_images = os.listdir(preprocessed_dir)

# Split the dataset into training and validation sets
train_images, val_images = train_test_split(preprocessed_images, test_size=0.2, random_state=42)

# Define paths for the training and validation image and label files
train_image_paths = [os.path.join(preprocessed_dir, img) for img in train_images]
train_label_paths = [os.path.join(labels_dir, img.replace(".jpg", ".txt")) for img in train_images]

val_image_paths = [os.path.join(preprocessed_dir, img) for img in val_images]
val_label_paths = [os.path.join(labels_dir, img.replace(".jpg", ".txt")) for img in val_images]

# Define YOLOv5 configuration
yolov5_config = f'''
train: {os.path.join(data_dir, 'train')}
val: {os.path.join(data_dir, 'val')}

nc: 1
names: ['license_plate']
'''

# Write the YOLOv5 configuration to file
yolov5_config_path = os.path.join(data_dir, 'yolov5_config.yaml')
with open(yolov5_config_path, 'w') as f:
    f.write(yolov5_config)

# Train the YOLOv5 model
import subprocess

# Define the training command
train_command = [
    'python', 'train.py',
    '--img', '640',
    '--batch', '16',
    '--epochs', '30',
    '--data', yolov5_config_path,
    '--cfg', 'models/yolov5s.yaml',
    '--weights', '']

# Run the training command
subprocess.run(train_command)


CompletedProcess(args=['python', 'train.py', '--img', '640', '--batch', '16', '--epochs', '30', '--data', '/content/drive/MyDrive/Project/LPR/preprocessed_images/yolov5_config.yaml', '--cfg', 'models/yolov5s.yaml', '--weights', ''], returncode=1)

In [11]:
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository


Cloning into 'yolov5'...
remote: Enumerating objects: 16656, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16656 (delta 1), reused 5 (delta 0), pack-reused 16649
Receiving objects: 100% (16656/16656), 15.07 MiB | 20.44 MiB/s, done.
Resolving deltas: 100% (11443/11443), done.


In [1]:
%cd yolov5
!pip install -r requirements.txt  # Install dependencies


/content/yolov5


In [2]:
!python train.py --img 640 --batch 16 --epochs 50 --data path/to/your/data.yaml --cfg models/yolov5s.yaml --weights '' --name your_experiment_name


2024-06-03 15:52:31.760765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 15:52:31.760862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 15:52:31.893660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=models/yolov5s.yaml, data=path/to/your/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, s

In [26]:
!pip install tesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=68210f2b0db1843ef0b8b86607d68bc1bc4fff26eb8d89bc0682799f28890306
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract


In [30]:
!apt-get update
!apt-get install -y tesseract-ocr


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu 

In [31]:
from models.experimental import attempt_load
from utils.general import non_max_suppression
from utils.augmentations import letterbox

# Define the directory containing the test images
test_images_dir = "/content/drive/MyDrive/Project/LPR/test"

# Load the trained YOLOv5 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attempt_load("/content/drive/MyDrive/Project/LPR/yolov5s.pt", device=device)
model.to(device).eval()

# Define scale_coords function
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coords (xyxy) from img1_shape to img0_shape
    if ratio_pad is None:  # calculate from img0_shape
        gain = max(img1_shape) / max(img0_shape)  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:  # use ratio and pad
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])  # x1
    coords[:, 1].clamp_(0, img0_shape[0])  # y1
    coords[:, 2].clamp_(0, img0_shape[1])  # x2
    coords[:, 3].clamp_(0, img0_shape[0])  # y2
    return coords

# Create empty lists to store results
image_names = []
plate_numbers = []

# Iterate through the test images
for img_name in os.listdir(test_images_dir):
    # Read the image
    img_path = os.path.join(test_images_dir, img_name)
    img = cv2.imread(img_path)
    assert img is not None, f"Image not found: {img_path}"

    # Preprocess image for YOLOv5
    img_resized = letterbox(img, new_shape=640)[0]
    img_resized = img_resized.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img_resized = np.ascontiguousarray(img_resized)

    # Convert image to tensor
    img_tensor = torch.from_numpy(img_resized).to(device).float()
    img_tensor /= 255.0  # Normalize
    if img_tensor.ndimension() == 3:
        img_tensor = img_tensor.unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        pred = model(img_tensor, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

    for det in pred:
        if len(det):
            # Rescale boxes from img_size to img0 size
            det[:, :4] = scale_coords(img_tensor.shape[2:], det[:, :4], img.shape).round()

            for *xyxy, conf, cls in reversed(det):
                if int(cls) == 0:  # Assuming class 0 represents license plates
                    # Extract plate coordinates
                    x_min, y_min, x_max, y_max = map(int, xyxy)

                    # Crop license plate from image
                    plate_img = img[y_min:y_max, x_min:x_max]

                    # Perform OCR on the cropped plate image
                    plate_text = pytesseract.image_to_string(plate_img, config='--psm 7')

                    # Clean extracted plate text
                    cleaned_plate_text = ''.join([char for char in plate_text if char.isalnum()])

                    # Store image name and plate number
                    image_names.append(img_name)
                    plate_numbers.append(cleaned_plate_text)

# Create DataFrame for plate numbers
plate_df = pd.DataFrame({
    'image_name': image_names,
    'plate_number': plate_numbers
})

# Save plate numbers to CSV
output_csv_path = '/content/drive/MyDrive/Project/LPR/plate_numbers.csv'
plate_df.to_csv(output_csv_path, index=False)

print('Detection and OCR results saved successfully.')


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


Detection and OCR results saved successfully.
